<h1>Multi-agent

multi-agent可用于处理一些single agent不好处理的问题<br>
两种常用的框架：supervisor/swarm<br>
supervisor：一个父agent管理几个子agent<br>
swarm：多个agent间相互作用互相影响

<h1>Supervisor</h1>

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

In [11]:
@tool
def book_hotel(hotel_name: str):
    """Book a hotel"""
    return f"Successfully booked a stay at {hotel_name}."
@tool
def book_flight(from_airport: str, to_airport: str):
    """Book a flight"""
    return f"Successfully booked a flight from {from_airport} to {to_airport}."


In [46]:
model = ChatOpenAI(
    model="gpt-3.5-turbo", 
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://free.v36.cm/v1"
)

In [12]:
#航班预订智能代理
flight_assistant = create_react_agent(
    model="gpt-3.5-turbo",
    tools=[book_flight],
    prompt="You are a flight booking assistant",
    name="flight_assistant"
)
#酒店预订智能代理
hotel_assistant = create_react_agent(
    model="gpt-3.5-turbo",
    tools=[book_hotel],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"
)

In [13]:
#创建主管系统，管理子系统
supervisor = create_supervisor(
    agents=[flight_assistant, hotel_assistant],
    model=ChatOpenAI(
    model="gpt-3.5-turbo", 
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://free.v36.cm/v1"
),
    prompt=(
        "You manage a hotel booking assistant and a"
        "flight booking assistant. Assign work to them."
    )
).compile()

In [ ]:
try:
    for chunk in supervisor.stream(
        {
            "messages": [
                {
                    "role": "user",
                    "content": """嘿，我想订一趟旅行！
                    我计划从波士顿飞往纽约，大概是7月15号。就一个人旅行，希望能订个早上的航班，最好能靠窗座位。
                    住宿方面，我看上了麦基特里克酒店。从7月15号住到18号，就一个人。如果能给我安排个非吸烟房间，最好还能是高层房间就太棒了！
                    麻烦帮我确认一下这些行程，谢谢！"""
                }
            ]
        }
    ):
        print(chunk)
        print("\n")
except Exception as e:
    print(f"An error occurred: {e}")

{'supervisor': {'messages': [HumanMessage(content='嘿，我想订一趟旅行！\n我计划从波士顿飞往纽约，大概是7月15号。就一个人旅行，希望能订个早上的航班，最好能靠窗座位。\n住宿方面，我看上了麦基特里克酒店。从7月15号住到18号，就一个人。如果能给我安排个非吸烟房间，最好还能是高层房间就太棒了！\n麻烦帮我确认一下这些行程，谢谢！', additional_kwargs={}, response_metadata={}, id='3f9a9aa6-f5fd-447d-a4d6-7f693939ba8e'), AIMessage(content='好的，我会将您的请求分配给航班预订助手和酒店预订助手。\n\n### 航班预订助手：\n- **出发地**：波士顿\n- **目的地**：纽约\n- **出发日期**：7月15号\n- **乘客人数**：1人\n- **航班时间**：早上\n- **座位偏好**：靠窗座位\n\n### 酒店预订助手：\n- **酒店名称**：麦基特里克酒店\n- **入住日期**：7月15号\n- **退房日期**：7月18号\n- **乘客人数**：1人\n- **房间类型**：非吸烟房间，最好是高层房间\n\n我将立即开始处理这些请求，并尽快给您确认信息。请稍等！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 250, 'prompt_tokens': 194, 'total_tokens': 444, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_0c321cec1fa7', 'id': 'chatcmpl-482c883f-f54c-4a1b-9196-7a760c94cf3a', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, name='superviso

可以看出正常调用了llm但设定的tools又被忽略了，怎么会事呢

<h1>Swarm</h1>

<h1>handoffs</h1><br>
可将智能体控制权转交为另一个智能体